## 调包实现贝叶斯分类器完成二分类

实验内容：
1. 数据预处理（删除异常值如sc_w=0的样本，将标签的四分类改为二分类，0和1归为一类，2和3归为一类）
2. 使用GaussianNB、BernoulliNB、MultinomialNB完成手机市场价分类
3. 计算各自十折交叉验证的精度、查准率、查全率、F1值
4. 根据精度、查准率、查全率、F1值的实际意义以及四个值的对比阐述三个算法在手机市场价分类中的表现对比

In [1]:
#数据预处理
import pandas as pd
import numpy as np
data = pd.read_csv('../data/mobile_phone/train.csv', delimiter=',')

data = data.drop(data[data['sc_w']==0].index)
print(len(data[data.sc_w == 0].index.tolist()))


0


In [2]:
data.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [3]:
def pre_func(x):
    if x==0 or x==1:
        return 0
    elif x==2 or x==3:
        return 1

data.price_range = data.price_range.apply(pre_func)

In [4]:

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [20]:
features = data.columns
features = list(set(features) - {"price_range"})
X = data[features]
y = data.price_range

In [21]:
X.shape

(1820, 20)

In [22]:
# 计算十折交叉验证下，GaussianNB、BernoulliNB、MultinomialNB的精度、查准率、查全率、F1值
result_data = []
models = [(GaussianNB(), "GaussianNB"), (BernoulliNB(), "BernoulliNB"), (MultinomialNB(), "MultinomialNB")]
for model, model_name in models:
    y_hat = cross_val_predict(model, X, y, cv=10)
    acc = accuracy_score(y, y_hat)
    precision = precision_score(y, y_hat)
    recall = recall_score(y, y_hat)
    f1 = f1_score(y, y_hat)
    result = [model_name, acc, precision, recall, f1]
    result_data.append(result)
columns = ["model", "acc", "precision", "recall", "f1"]
pd.DataFrame(data=result_data, columns=columns)

,model,acc,precision,recall,f1
0,GaussianNB,0.933516,0.933551,0.934569,0.934060
1,BernoulliNB,0.492857,0.497180,0.576881,0.534074
2,MultinomialNB,0.809890,0.782394,0.862595,0.820539


表格见上方输出

## 手动实现高斯朴素贝叶斯分类器
1. 实现高斯朴素贝叶斯分类器并完成手机市场价分类，数据集采用6：4划分
2. 计算模型的查准率，查全率，F1值

我们要实现一个可以处理连续特征的，服从高斯分布的朴素贝叶斯分类器。

### 符号

给定训练集 $T$

$$T = \{(x_1, y_1), (x_2, y_2), ···, (x_N, y_N)\}$$

其中，$x$ 为样本的特征，$y$ 是该样本对应的标记，下标表示对应的是第几个样本，上标表示第几个特征。训练集 $T$ 内一共 $\vert T \vert = N$ 个样本。

假设我们的任务是处理 $K$ 类分类任务，记类标记分别为 $c_1, c_2, ..., c_k$ 。

### 目标

我们的目标是对样本进行分类，这里我们用概率的方法，求 $P(Y = c_k \mid X = x), \ k = 1, 2, ..., K$ 中最大的那个概率对应的 $k$ 是哪个，也就是，给定样本 $x$ ，模型认为它是哪个类别的概率最大。

### 原理

由贝叶斯公式：

$$
\begin{aligned}
    P(Y = c_k \mid X = x) &= \frac{P(Y = c_k, X = x)}{P(X = x)} \\
                  &= \frac{P(X = x \mid Y = c_k)P(Y = c_k)}{\sum_kP(X = x \mid Y = c_k)P(Y = c_k)} \\
\end{aligned}
$$

这里，我们要求 $K$ 个概率中最大的那个，而这 $K$ 个概率的分母都相同，我们可以忽略分母部分，比较分子部分的大小，也就是比较 **先验概率** $P(Y = c_k)$ 和 **似然** $P(X = x \mid Y = c_k)$ 的乘积。

通过先验概率分布

$$
P(Y = c_k), \ k = 1, 2, ..., K
$$

和条件概率分布

$$
P(X = x \mid Y = c_k) = P(X^{(1)} = x^{(1)}, ···, X^{(n)} = x^{(n)} \mid Y = c_k), \ k = 1, 2, ..., K
$$

我们就可以得到联合概率分布 $P(X = x, Y = c_k)$ 。

**那么，问题就转化为了，如何求先验概率和似然？**

#### 1. 先验概率 $P(Y = c_k)$ ：

先验概率的求解很简单，只要统计训练集中类别 $k$ 出现的概率即可。

$$
P(Y = c_k) = \frac{\mathrm{number} \ \mathrm{of}\ c_k}{N}
$$

#### 2. 似然 $P(X = x \mid Y = c_k)$ ：

求解这个条件概率比较复杂，**这里我们要假设特征之间相互独立**，可得

$$P(X = x \mid Y = c_k) = \prod^n_{j=1}P(X^{(j)}=x^{(j)} \mid Y = c_k)$$

其中， $x^{(j)}$ 表示样本 $x$ 的第 $j$ 个特征。

这样，复杂的条件概率就转换为了多个特征条件概率的乘积。v

#### 3. 特征 $j$ 的条件概率 $P(X^{(j)}=x^{(j)} \mid Y = c_k)$ ：

因为我们处理的特征都是连续型特征，一般我们假设这些特征服从正态分布。

当 $Y = c_k$ 时，$X^{(j)} = a_{jl}$ 的概率可由下面的公式计算得到：

$$
P(X^{(j)} = a_{jl} \mid Y = c_k) = \frac{1}{\sqrt{2 \pi \sigma^2_{c_k,j}}} \exp{\bigg( - \frac{(a_{jl} - \mu_{c_k,j})^2}{2 \sigma^2_{c_k,j}} \bigg)}
$$

这里 $\mu_{c_k,j}$ 和 $\sigma^2_{c_k,j}$ 分别表示当 $Y = c_k$ 时，第 $j$ 个特征的均值和方差，**这个均值和方差都是通过训练集的样本计算出来的**。

因为正态分布只需要两个参数（均值和方差）就可以确定，对于特征 $j$ 我们要估计 $K$ 个类别的均值和方差，所以特征 $j$ 的参数共有 $2K$个。

### 综上

朴素贝叶斯分类器可以表示为：

$$
y = \mathop{\arg\max}_{c_k} P(Y = c_k) \prod_j P(X^{(j)} = x^{(j)} \mid Y = c_k)
$$

### 实现
实现的时候会遇到数值问题，在上面的条件概率连乘中，如果有几个概率值很小，它们的连乘就会导致下溢，解决方案就是将其改写为连加的形式。

首先，我们的目标是：

$$
y = \mathop{\arg\max}_{c_k} P(Y = c_k) \prod_j P(X^{(j)} = x^{(j)} \mid Y = c_k)
$$

比较这 $K$ 个数值的大小，然后取最大的那个数对应的 $k$。

为了解决可能出现的下溢问题，我们对上面的式子取对数，因为是对 $K$ 项都取对数，不会改变单调性，所以取对数是不影响它们之间的大小关系的。

那目标就变成了：

$$
\begin{aligned}
y &= \mathop{\arg\max}_{c_k} \big[ \log^{ \ P(Y = c_k) \prod_j P(X^{(j)} = x^{(j)} \mid Y = c_k)} \big] \\
&= \mathop{\arg\max}_{c_k} \big[ \log^{ \ P(y = c_k)} + \sum_j \log^{ \ P(X^{(j)} = x^{(j)} \mid Y = c_k)} \big]
\end{aligned}
$$

在求条件概率的时候，也进行变换：

$$\begin{aligned}
\log^{ \ P(X^{(j)} = x^{(j)} \mid Y = c_k)} &= \log^{ \ \bigg[\frac{1}{\sqrt{2 \pi \sigma^2_{c_k,j}}} \exp{\bigg(- \frac{(a_{jl} - \mu_{c_k,j})^2}{2 \sigma^2_{c_k,j}}\bigg)}\bigg]}\\
&= \log^{ \frac{1}{\sqrt{2 \pi \sigma^2_{c_k,j}}} } + \log^{ \exp{\bigg(- \frac{(a_{jl} - \mu_{c_k,j})^2}{2 \sigma^2_{c_k,j}}\bigg)} }\\
&= - \frac{1}{2} \log^{2 \pi \sigma^2_{c_k,j}} - \frac{1}{2} \frac{(a_{jl} - \mu_{c_k,j})^2}{\sigma^2_{c_k,j}}
\end{aligned}
$$

所以，高斯朴素贝叶斯就可以变形为：

$$
y = \mathop{\arg\max}_{c_k} \bigg[ \log^{ \ P(y = c_k)} + \sum_j \big( - \frac{1}{2} \log^{2 \pi \sigma^2_{c_k,j}} - \frac{1}{2} \frac{(a_{jl} - \mu_{c_k,j})^2}{\sigma^2_{c_k,j}} \big) \bigg]
$$

上式就是我们需要求的，我们要求出 $K$ 个值，然后求最大的那个对应的 $k$。

**接下来我们开始实现高斯朴素贝叶斯**，我们以类的形式实现这个高斯朴素贝叶斯。因为朴素贝叶斯是懒惰学习，所以这个模型只有在预测的时候，会进行大量的运算。

In [23]:
class myGaussianNB:
    '''
    处理连续特征的高斯朴素贝叶斯
    '''
    def __init__(self):
        '''
        初始化四个字典
        self.label_mapping     类标记 与 下标(int)
        self.probability_of_y  类标记 与 先验概率(float)
        self.mean              类标记 与 均值(np.ndarray)
        self.var               类标记 与 方差(np.ndarray)
        '''
        self.label_mapping = dict()
        self.probability_of_y = dict()
        self.mean = dict()
        self.var = dict()
        
    def _clear(self):
        '''
        为了防止一个实例反复的调用fit方法，我们需要每次调用fit前，将之前学习到的参数删除掉
        '''
        self.label_mapping.clear()
        self.probability_of_y.clear()
        self.mean.clear()
        self.var.clear()
    
    def fit(self, trainX, trainY):
        '''
        这里，我们要根据trainY内的类标记，针对每类，计算这类的先验概率，以及这类训练样本每个特征的均值和方差

        Parameters
        ----------
            trainX: np.ndarray, 训练样本的特征, 维度：(样本数, 特征数)
        
            trainY: np.ndarray, 训练样本的标记, 维度：(样本数, )
        '''
        
        # 先调用_clear
        self._clear()
        
        # 获取类标记
        labels = np.unique(trainY)
        
        # 添加类标记与下标的映射关系
        self.label_mapping = {label: index for index, label in enumerate(labels)}
        
        # 遍历每个类
        for label in labels:
            
            # 取出为label这类的所有训练样本，存为 x
            x = trainX[trainY == label]
            x = x.to_numpy()
            
            # 计算先验概率，用 x 的样本个数除以训练样本总个数，存储到 self.probability_of_y 中，键为 label，值为先验概率
            
            self.probability_of_y[label] = len(x) / len(trainY)
            
            # 对 x 的每列求均值，使用 keepdims = True 保持维度，存储到 self.mean 中，键为 label，值为每列的均值组成的一个二维 np.ndarray
            # YOUR CODE HERE
            self.mean[label] = np.mean(x, axis=0, keepdims=True)
            # print(self.mean[label].shape)
            
            
            # 这句话是debug用的，如果不满足下面的条件，会直接跳出
            assert self.mean[label].shape == (1, trainX.shape[1])
            
            # 对 x 的每列求方差，使用 keepdims = True 保持维度，存储到 self.var 中，键为 label，值为每列的方差组成的一个二维 np.ndarray
            self.var[label] = np.var(x, axis=0, keepdims=True)

            # debug
            assert self.var[label].shape == (1, trainX.shape[1])
            
            # 平滑，因为方差在公式的分母部分，我们要加一个很小的数，防止除以0
            self.var[label] += 1e-9 * np.var(trainX, axis = 0).max()
        
    def predict(self, testX):
        '''
        给定测试样本，预测测试样本的类标记，这里我们要实现化简后的公式

        Parameters
        ----------
            testX: np.ndarray, 测试的特征, 维度：(测试样本数, 特征数)
    
        Returns
        ----------
            prediction: np.ndarray, 预测结果, 维度：(测试样本数, )
        '''
        
        # 初始化一个空矩阵 results，存储每个样本属于每个类的概率，维度是 (测试样本数，类别数)，每行表示一个样本，每列表示一个特征
        results = np.empty((testX.shape[0], len(self.probability_of_y)))
        
        # 初始化一个列表 labels，按 self.label_mapping 的映射关系存储所有的标记，一会儿会在下面的循环内部完成存储
        labels = [0] * len(self.probability_of_y)
        
        # 遍历当前的类，label为类标记，index为下标，我们将每个样本预测出来的这个 label 的概率，存到 results 中的第 index 列
        for label, index in self.label_mapping.items():
            
            # 先验概率存为 py
            py = self.probability_of_y[label]
            
            # 使用变换后的公式，计算所有特征的条件概率之和，存为sum_of_conditional_probability
            # self.var mean per label has vars as shape is len(x)
            sum_of_conditional_probability = np.sum(-0.5*(np.log(2*np.pi*self.var[label]) + np.square(testX - self.mean[label])/self.var[label]), axis=1)
            # YOUR CODE HERE
            
            # debug
            assert sum_of_conditional_probability.shape == (len(testX), )
            
            # 使用变换后的公式，将 条件概率 与 log先验概率 相加，存为result，维度应该是 (测试样本数, )
            
            result = np.log(py) + sum_of_conditional_probability
            # YOUR CODE HERE
            
            # debug
            assert result.shape == (len(testX), )
            
            # 将所有测试样本属于当前这类的概率，存入到results中
            results[:, index] = result
            
            # 将当前的label，按index顺序放入到labels中
            labels[index] = label
        
        # 将labels转换为np.ndarray
        np_labels = np.array(labels)
        
        # 循环结束后，就计算出了给定测试样本，当前样本属于这类的概率的近似值，存放在了results中，每行对应一个样本，每列对应一个特征
        # 我们要求每行的最大值对应的下标，也就是求每个样本，概率值最大的那个下标是什么，结果存入max_prob_index中

        max_prob_index = np.argmax(results, axis = 1)
        
        # debug
        assert max_prob_index.shape == (len(testX), )
        
        # 现在得到了每个样本最大概率对应的下标，我们需要把这个下标变成 np_labels 中的标记
        prediction = np_labels[max_prob_index]
        
        # debug
        assert prediction.shape == (len(testX), )
        
        # 返回预测结果
        return prediction

In [24]:
import pandas as pd
import numpy as np
data = pd.read_csv('../data/mobile_phone/train.csv', delimiter=',')

In [25]:
data = data.drop(data[data['sc_w']==0].index)

def pre_func(x):
    if x==0 or x==1:
        return 0
    elif x==2 or x==3:
        return 1

data.price_range = data.price_range.apply(pre_func)
features = data.columns
features = list(set(features) - {"price_range"})
X = data[features]
y = data.price_range

In [26]:
from sklearn.model_selection import train_test_split
# your code here
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.6)

In [27]:
model = myGaussianNB()
# 计算模型指标
model.fit(x_train, y_train)
y_hat = model.predict(x_test)
print("acc", accuracy_score(y_test, y_hat))
print("precision", precision_score(y_test, y_hat))
print("recall_score", recall_score(y_test, y_hat))
print("f1", f1_score(y_test, y_hat))


acc 0.9491758241758241
precision 0.9373368146214099
recall_score 0.9650537634408602
f1 0.9509933774834438


## 实现带有拉普拉斯修正的朴素贝叶斯

实验内容：
1. 叙述拉普拉斯修正的作用
2. 使用给定的数据集
3. 给出实现的代码，要有详细的注释
4. 给出模型评价指标的结果

## 拉普拉斯修正的具体介绍
在训练集中总共的分类数，用 N 表示；di 属性可能的取值数用 Ni 表示，因此原来的先验概率 P(c) 的计算公式由：

P(c) = Dc / D

被拉普拉斯修正为

P(c) = (Dc + 1) / (D + N)

类的条件概率由P(xi|c) = Dc,xi / Dc

被拉普拉斯修正为

P(xi|c) = (Dc,xi + 1) / (Dc + Ni)


### Balance Scale Data Set 离散特征的三分类数据集
#### Attribute Information:
	1. Class Name: 3 (L（左边重）, B（天平平衡）, R（右边重）) 
	2. Left-Weight: 5 (1, 2, 3, 4, 5)
	3. Left-Distance: 5 (1, 2, 3, 4, 5)
	4. Right-Weight: 5 (1, 2, 3, 4, 5)
	5. Right-Distance: 5 (1, 2, 3, 4, 5)

In [2]:
import numpy as np
data = np.loadtxt("./balance-scale.data", str, unpack = True)
# print(data)
#数据处理：分类结果数值化，B->0, L->-1, R->1
dicts = {'R': 1, 'L': -1, 'B': 0}
# print(dicts['R'])
results = [] 
for i in data:
    tmp = []
    for j in i:
        if j == ',':
            continue
        if j == 'B' or j == 'R' or j == 'L':
            tmp.append(dicts[j])
        else:
            tmp.append(int(j))
    results.append(tmp)
data = np.array(results) # 处理后的数据集
datax = data[ : , 1 : ]
datay = data[ : , 0]
print(datax)

[[1 1 1 1]
 [1 1 1 2]
 [1 1 1 3]
 ...
 [5 5 5 3]
 [5 5 5 4]
 [5 5 5 5]]


In [3]:
#划分数据集测试集30%，训练集70%
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(datax, datay, test_size = 0.3, random_state = 32)
trainX.shape, trainY.shape, testX.shape, testY.shape

((437, 4), (437,), (188, 4), (188,))

### 多分类基线

In [5]:
# 多分类基线
clf = GaussianNB()
clf.fit(trainX, trainY)
y_hat = clf.predict(testX)
print("acc", accuracy_score(testY, y_hat))
print("precision", precision_score(testY, y_hat, average="weighted"))
print("recall_score", recall_score(testY, y_hat, average="weighted"))
print("f1", f1_score(testY, y_hat, average="weighted"))

acc 0.8936170212765957
precision 0.8231104528651759
recall_score 0.8936170212765957
f1 0.8567486906207958


D:\Program Files (x86)\python3.10\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
class myGaussianNB:
    '''
    处理离散特征的高斯朴素贝叶斯
    '''
    def __init__(self):
        '''
        初始化四个字典
        self.label_mapping     类标记 与 下标(int)
        self.probability_of_y  类标记 与 先验概率(float)
        self.probability_of_attributes 类标记 与 每个属性的类条件概率
        '''
        self.label_mapping = dict()
        self.probability_of_y = dict()
        self.probability_of_attributes = dict()
        
    def _clear(self):
        '''
        为了防止一个实例反复的调用fit方法，我们需要每次调用fit前，将之前学习到的参数删除掉
        '''
        self.label_mapping.clear()
        self.probability_of_y.clear()
        self.probability_of_attributes.clear()
    
    def fit(self, trainX, trainY):
        '''
        这里，我们要根据trainY内的类标记，针对每类，计算这类的先验概率，以及这类训练样本每个特征的均值和方差

        Parameters
        ----------
            trainX: np.ndarray, 训练样本的特征, 维度：(样本数, 特征数)
        
            trainY: np.ndarray, 训练样本的标记, 维度：(样本数, )
        '''
        
        # 先调用_clear
        self._clear()
        
        # 获取类标记
        labels = np.unique(trainY)
        # print("labels = ", labels)
        
        # 添加类标记与下标的映射关系
        self.label_mapping = {label: index for index, label in enumerate(labels)}
        # print(self.label_mapping)
        
        # 遍历每个类
        for label in labels:
            
            # 取出为label这类的所有训练样本，存为 x

            x = trainX[trainY == label, :]
            # x = x.to_numpy()
            
            # 计算先验概率，用 x 的样本个数除以训练样本总个数，存储到 self.probability_of_y 中，键为 label，值为先验概率
            self.probability_of_y[label] = (len(x) +1)/ (6 + len(trainY))
            
            # 计算属性的类条件概率
            result = dict()
            for index in range(trainX.shape[1]):
                tmp = []
                for i in range(1, 6): # 每个属性的取值范围：{1, 2, 3, 4, 5}
                    total = [t for t in x[ : , index] if t == i]
                    # 拉普拉斯修正
                    probability = (len(total) + 1) / (6+len(x))

                    tmp.append(probability)
                result[index] = tmp
            self.probability_of_attributes[label] = result
            # print('self.probability_of_attributes = ', self.probability_of_attributes[label])
            
        
    def predict(self, testX):
        '''
        给定测试样本，预测测试样本的类标记，这里我们要实现化简后的公式

        Parameters
        ----------
            testX: np.ndarray, 测试的特征, 维度：(测试样本数, 特征数)
    
        Returns
        ----------
            prediction: np.ndarray, 预测结果, 维度：(测试样本数, )
        '''
        
        # 初始化一个空矩阵 results，存储每个样本属于每个类的概率，维度是 (测试样本数，类别数)，每行表示一个样本，每列表示一个特征
        # testX = testX.to_numpy()
        results = np.empty((testX.shape[0], len(self.probability_of_y)))
        
        # 初始化一个列表 labels，按 self.label_mapping 的映射关系存储所有的标记，一会儿会在下面的循环内部完成存储
        labels = [0] * len(self.probability_of_y)
        
        # 遍历当前的类，label为类标记，index为下标，我们将每个样本预测出来的这个 label 的概率，存到 results 中的第 index 列
        for label, index in self.label_mapping.items():
            
            # print("label = ", label)
            
            # 先验概率存为 py
            py = self.probability_of_y[label]
            
            #  计算测试样本是label类的概率
            result = []
            for i in range(testX.shape[0]):
                tmp = []
                for j in range(testX.shape[1]):
                    # print('j = ', j, 'testX[i][j] = ', testX[i][j])

                    tmp.append(self.probability_of_attributes[label][j][testX[i][j] - 1]) # 细节之处: 越界
                result.append(np.sum(np.log(tmp)) + np.log(py)) # 取对数计算防止下溢
                
            # 存入到results中的第index列中
            results[:, index] = result
            
            # 将当前的label，按index顺序放入到labels中
            labels[index] = label
        
        # 将labels转换为np.ndarray
        np_labels = np.array(labels)
        
        # 循环结束后，就计算出了给定测试样本，当前样本属于这类的概率的近似值，存放在了results中，每行对应一个样本，每列对应一个特征
        # 我们要求每行的最大值对应的下标，也就是求每个样本，概率值最大的那个下标是什么，结果存入max_prob_index中
        max_prob_index = np.argmax(results, axis = 1)
        
        # debug
        assert max_prob_index.shape == (len(testX), )
        
        # 现在得到了每个样本最大概率对应的下标，我们需要把这个下标变成 np_labels 中的标记
        
        prediction = np_labels[max_prob_index]
        
        # debug
        assert prediction.shape == (len(testX), )
        
        # 返回预测结果
        return prediction

In [31]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

model = myGaussianNB()
# your code here
print("带拉普拉斯修正的朴素贝叶斯分类器在测试集上的四项指标：")
model.fit(trainX, trainY)
y_hat = model.predict(testX)
print("acc", accuracy_score(testY, y_hat))
print("precision", precision_score(testY, y_hat, average="weighted"))
print("recall_score", recall_score(testY, y_hat, average="weighted"))
print("f1", f1_score(testY, y_hat, average="weighted"))

带拉普拉斯修正的朴素贝叶斯分类器在测试集上的四项指标：
acc 0.9042553191489362
precision 0.8324468085106383
recall_score 0.9042553191489362
f1 0.8668015094379055


D:\Anaconda3_2022.05\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 实验心得 踩坑记录
一开始，遇到了四个评价指标全是1.0。
认为是代码出错了，最后定位到数据预处理部分出错。feature和label划分出现问题。feature中没有把label去掉，其实是对set的特性不熟。一开始使用的是`set("price_range")`。
但是这样是把整个串的所有单个字符做集合，后边修改为`{"price_range"}`就是把整个串作为集合的一个元素。